In [2]:
import pandas as pd
import numpy as np
import mrjob
import nltk as nlp
from mrjob.job import MRJob
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
import json

In [38]:
WORD_RE = re.compile(r'[\w]+')
with open('stopwords.txt') as f:
    lines = f.readlines()

STOP_WORDS = [n.replace("\n", "") for n in lines]
delimiters = ['(', ')', ']', '[', ']', '{', '}', '.', '!', '?', ',', ';',':','+','=','-','_','"','`','~','#','@','&','*','%','€','$','§','/']

In [275]:
df = pd.read_json("reviews_devset.json", lines=True)

In [276]:
df.shape

(78829, 10)

In [115]:
json_set = json.loads(df.to_json(orient='records'))
words = {}

for i in range(78829):
    text = json_set[i]['reviewText']
    for word in WORD_RE.findall(text):
        if len(word) > 1 and word not in STOP_WORDS and any(x in word for x in delimiters) == False:
            if (word.lower() not in words.keys()):
                words[word.lower()] = 1
            else:
                words[word.lower()] += 1

In [116]:
json_set = json.loads(df.to_json(orient='records'))
words2 = {}

for i in range(78829):
    text = json_set[i]['reviewText']
    category = json_set[i]['category']
    for word in WORD_RE.findall(text):
        if len(word) > 1 and word not in STOP_WORDS and any(x in word for x in delimiters) == False:
            if (category not in words2.keys()):
                words2[category] = {}
            else:
                if (word.lower() not in words2[category].keys()):
                    words2[category][word.lower()] = 1
                else:
                    words2[category][word.lower()] += 1

In [183]:
words2

{'Patio_Lawn_and_Garde': {'gift': 23,
  'husband': 22,
  'he': 16,
  'making': 24,
  'things': 35,
  'time': 171,
  'love': 91,
  'food': 33,
  'directions': 17,
  'simple': 17,
  'easy': 177,
  'interpret': 1,
  'make': 71,
  'we': 116,
  'kinds': 5,
  'cuisine': 1,
  'raichlen': 1,
  'recipes': 1,
  'barbecue': 1,
  'trail': 4,
  'calls': 4,
  'get': 9,
  'open': 14,
  'page': 3,
  'have': 19,
  'you': 52,
  'provided': 8,
  'insight': 1,
  'culture': 1,
  'produced': 4,
  'it': 390,
  'broadening': 1,
  'horizons': 1,
  'yum': 1,
  'this': 292,
  'nice': 70,
  'spreader': 2,
  'feels': 8,
  'solid': 23,
  'pneumatic': 1,
  'tires': 6,
  'give': 28,
  'great': 248,
  'maneuverability': 1,
  'handling': 3,
  'bumps': 1,
  'the': 432,
  'control': 29,
  'arm': 6,
  'metal': 30,
  'cable': 6,
  'precise': 1,
  'long': 73,
  'settings': 9,
  'experimentation': 1,
  'products': 30,
  'true': 12,
  'good': 212,
  'distribution': 2,
  'flings': 1,
  'material': 22,
  'farther': 2,
  'side':

In [168]:
words2['Apps_for_Android']['games']

329

In [171]:
claculator = 0
for i in words2.keys():
    if ("games" in words2[i].keys()):
        claculator += words2[i]["games"]

In [172]:
claculator

902

In [173]:
len(words2['Apps_for_Android'].keys())

6618

In [225]:
all_summ = 0
for i in words2['Apps_for_Android'].keys():
    all_summ += words2['Apps_for_Android'][i]

In [226]:
all_summ

36081

In [198]:
sumall = 0
for i in list(my_dat.target):
    sumall += int(my_dat[my_dat.target == i].sum(axis=1))
    

/tmp/ipykernel_4465/782954207.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  sumall += int(my_dat[my_dat.target == i].sum(axis=1))


In [199]:
sumall

1816531

In [196]:
my_dat[my_dat.target == "Apps_for_Android"].sum(axis=1)

/tmp/ipykernel_4465/4059479131.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  my_dat[my_dat.target == "Apps_for_Android"].sum(axis=1)


1    36081
dtype: int64

In [117]:
columns = []
for i in words2.keys():
    [columns.append(w) for w in list(words2[i].keys()) if w not in columns]


In [135]:
my_dat = pd.DataFrame(columns=columns)

In [136]:
words2.keys()

dict_keys(['Patio_Lawn_and_Garde', 'Apps_for_Android', 'Book', 'Toys_and_Game', 'Office_Product', 'Digital_Music', 'Sports_and_Outdoor', 'Automotive', 'Beauty', 'Musical_Instrument', 'CDs_and_Vinyl', 'Kindle_Store', 'Clothing_Shoes_and_Jewelry', 'Electronic', 'Home_and_Kitche', 'Cell_Phones_and_Accessorie', 'Pet_Supplie', 'Movies_and_TV', 'Baby', 'Tools_and_Home_Improvement', 'Grocery_and_Gourmet_Food', 'Health_and_Personal_Care'])

In [153]:
my_dat['target'] = list(words2.keys())

In [138]:
my_dat = my_dat.replace(np.NaN, 0)

In [149]:
words2.keys()

dict_keys(['Patio_Lawn_and_Garde', 'Apps_for_Android', 'Book', 'Toys_and_Game', 'Office_Product', 'Digital_Music', 'Sports_and_Outdoor', 'Automotive', 'Beauty', 'Musical_Instrument', 'CDs_and_Vinyl', 'Kindle_Store', 'Clothing_Shoes_and_Jewelry', 'Electronic', 'Home_and_Kitche', 'Cell_Phones_and_Accessorie', 'Pet_Supplie', 'Movies_and_TV', 'Baby', 'Tools_and_Home_Improvement', 'Grocery_and_Gourmet_Food', 'Health_and_Personal_Care'])

In [147]:
for i in words2.keys():
    keys = words2[i].keys()
    for k in keys:
        my_dat.loc[my_dat["target"] == i, k] = words2[i][k]

In [156]:
c, p = chi2(my_dat.drop("target", axis=1), my_dat.target)

In [231]:
c[672]

5577.249999999998

In [206]:
list(my_dat.columns).index("games")

671

In [215]:
len(my_dat.columns)

102897

In [243]:
json_set[0]

{'reviewerID': 'A3EU2N0FAAJJU7',
 'asin': '044042321X',
 'reviewerName': 'Jeremy Ray',
 'helpful': [1, 2],
 'reviewText': "I've read all the books and seen all the latest episodes on HBO.they do a great job sticking to the book. and the art is awesome. If you don't want to read the thick books, this is perfect for you.  I am a nerd when it comes to games of thrones and really looking forward to the graphic novel when they get to the other books.",
 'overall': 5,
 'summary': 'love it',
 'unixReviewTime': 1378598400,
 'reviewTime': '09 8, 2013',
 'category': 'Book'}

In [292]:
json_set = json.loads(df.to_json(orient='records'))
words3 = {}

for i in range(78829):
    text = json_set[i]['reviewText']
    category = json_set[i]['category']
    for word in WORD_RE.findall(text):
        if len(word) > 1 and word.lower() not in STOP_WORDS and any(x in word for x in delimiters) == False and not word.isdigit():
            if (word.lower() not in words3.keys()):
                words3[word.lower()] = {category: 1}
            else:
                if (category not in words3[word.lower()].keys()):
                    words3[word.lower()][category] = 1
                else:
                    words3[word.lower()][category] += 1


In [295]:
calc = 0
for i in words3['games'].keys():
    calc += words3['games'][i]

In [299]:
calc

902

In [300]:
words3

{'gift': {'Patio_Lawn_and_Garde': 23,
  'Apps_for_Android': 1,
  'Book': 509,
  'Toys_and_Game': 179,
  'Office_Product': 22,
  'Digital_Music': 7,
  'Sports_and_Outdoor': 104,
  'Automotive': 9,
  'Beauty': 38,
  'Musical_Instrument': 9,
  'CDs_and_Vinyl': 60,
  'Kindle_Store': 19,
  'Clothing_Shoes_and_Jewelry': 256,
  'Electronic': 136,
  'Home_and_Kitche': 162,
  'Cell_Phones_and_Accessorie': 42,
  'Pet_Supplie': 4,
  'Movies_and_TV': 83,
  'Baby': 41,
  'Tools_and_Home_Improvement': 21,
  'Grocery_and_Gourmet_Food': 45,
  'Health_and_Personal_Care': 27},
 'husband': {'Patio_Lawn_and_Garde': 22,
  'Apps_for_Android': 10,
  'Book': 521,
  'Toys_and_Game': 47,
  'Office_Product': 8,
  'Digital_Music': 11,
  'Sports_and_Outdoor': 88,
  'Automotive': 10,
  'Beauty': 34,
  'Musical_Instrument': 5,
  'CDs_and_Vinyl': 27,
  'Kindle_Store': 56,
  'Clothing_Shoes_and_Jewelry': 161,
  'Electronic': 75,
  'Home_and_Kitche': 110,
  'Cell_Phones_and_Accessorie': 53,
  'Pet_Supplie': 11,
  'Movi

In [303]:
columns = []
for i in words3.keys():
    [columns.append(w) for w in list(words3[i].keys()) if w not in columns]

In [304]:
my_dat1 = pd.DataFrame(columns=columns)

In [305]:
my_dat1['target'] = list(words3.keys())

In [306]:
my_dat1 = my_dat1.replace(np.NaN, 0)

In [308]:
for i in words3.keys():
    keys = words3[i].keys()
    for k in keys:
        my_dat1.loc[my_dat1["target"] == i, k] = words3[i][k]

In [309]:
my_dat1

,Patio_Lawn_and_Garde,Apps_for_Android,Book,Toys_and_Game,Office_Product,Digital_Music,Sports_and_Outdoor,Automotive,Beauty,Musical_Instrument,...,Electronic,Home_and_Kitche,Cell_Phones_and_Accessorie,Pet_Supplie,Movies_and_TV,Baby,Tools_and_Home_Improvement,Grocery_and_Gourmet_Food,Health_and_Personal_Care,target
0,23,1,509,179,22,7,104,9,38,9,...,136,162,42,4,83,41,21,45,27,gift
1,22,10,521,47,8,11,88,10,34,5,...,75,110,53,11,109,42,36,42,90,husband
2,24,22,853,58,30,19,49,18,30,8,...,112,160,42,20,194,22,39,33,41,making
3,35,71,2114,96,57,19,92,30,51,16,...,346,166,68,32,296,43,51,34,105,things
4,171,403,4976,355,216,150,485,169,319,60,...,1375,741,477,177,1170,178,323,151,501,time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,awkwardbottom
100729,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,yacon
100730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,chlorophyl
100731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,forskolin


In [310]:
c, p = chi2(my_dat1.drop("target", axis=1), my_dat1.target)

MemoryError: Unable to allocate 75.6 GiB for an array with shape (100733, 100733) and data type int64

## Group by

In [ ]:
df['text'] = df.groupby(['category'])['reviewText'].transform(lambda x: ','.join(x))
df_2 = df.drop_duplicates('category')

In [ ]:
df_2 = df_2.set_index("category").drop(["reviewerID", "asin", "reviewerName", "helpful", "overall", "summary", "unixReviewTime", "reviewTime", "reviewText"], axis=1)

In [55]:
def prep_sentance(sent):

    
    word_tokens = re.sub(r'[^\w\s]', '', sent).split()

    filtered_sentence = [w for w in word_tokens if not w.lower() in lines]
    

    return print(filtered_sentence)

In [57]:
prep = []
for row in df2.reviewText:
    prep.append(prep_sentance(row))

SyntaxError: unexpected EOF while parsing (647158735.py, line 3)

In [51]:
prep_sentance(df2.reviewText[8])

['purchased', 'leaf', 'hog', 'year', 'impressed', 'helped', 'reduce', 'time', 'care', 'leaves', '80', 'hours', 'work', '35', 'hours', 'back', 'didnt', 'hurt', 'year', 'year', 'leaf', 'collection', 'system', 'time', 'spent', 'year', 'dumping', 'leaf', 'collection', 'bag', 'mulch', 'garbage', 'full', '4', '55', 'gallon', 'garbage', 'cans', 'started', 'mulching', 'big', 'piles', 'mulch', 'full', 'move', 'fabric', 'continue', 'sped', 'time', 'finishing', '20', 'hours', 'yeari', 'highly', 'recommend', 'mulch', 'single', 'bag', 'full', 'saved', 'ton', 'time', 'doubt']


In [2]:
def word_count(str):
    import operator
    counts = dict()

    for word in str:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    sorted_d = dict( sorted(counts.items(), key=operator.itemgetter(1),reverse=True))

    return sorted_d

In [3]:
my_dic = word_count(filtered_sentence)

NameError: name 'filtered_sentence' is not defined

In [5]:
df['reviewText'] = df['reviewText'].apply(prep_sentance)

NameError: name 'prep_sentance' is not defined

In [ ]:
vect = CountVectorizer()

In [ ]:
x = vect.fit_transform(df.reviewText)

In [ ]:
x.toarray()

MemoryError: Unable to allocate 60.5 GiB for an array with shape (78829, 103080) and data type int64

In [ ]:
class TextPreprocess(MRJob):

    def mapper(self, _, line):
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1

    def reducer(self, key, values):
        yield key, sum(values)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/teodor98/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True